## Bertram with portuguese-based-bert

### helper and processing

In [ ]:
!git clone https://github.com/H-TayyarMadabushi/AStitchInLanguageModels.git

Cloning into 'AStitchInLanguageModels'...
remote: Enumerating objects: 1030, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 1030 (delta 11), reused 4 (delta 4), pack-reused 1013
Receiving objects: 100% (1030/1030), 79.59 MiB | 32.19 MiB/s, done.
Resolving deltas: 100% (394/394), done.


In [ ]:
# %cd AStitchInLanguageModels/dependencies/sentence-transformers
# !pip install -e . 
# %cd /content/

In [ ]:
!git clone https://github.com/huggingface/transformers.git
%cd transformers/
!pip install --editable .
%cd /content/ 

Cloning into 'transformers'...
remote: Enumerating objects: 93680, done.
remote: Counting objects: 100% (233/233), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 93680 (delta 164), reused 191 (delta 134), pack-reused 93447
Receiving objects: 100% (93680/93680), 85.89 MiB | 14.90 MiB/s, done.
Resolving deltas: 100% (68686/68686), done.
/content/transformers
Obtaining file:///content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 77 kB 3.2 MB/s 
     |████████████████████████████████| 6.6 MB 10.8 MB/s 
     |████████████████████████████████| 596 kB 68.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Running setup.py develop for transformers
/content


In [ ]:
!pip install datasets
!pip install transformers
!pip install sentence_transformers

     |████████████████████████████████| 325 kB 5.0 MB/s 
     |████████████████████████████████| 1.1 MB 52.2 MB/s 
     |████████████████████████████████| 212 kB 50.9 MB/s 
     |████████████████████████████████| 136 kB 51.3 MB/s 
     |████████████████████████████████| 127 kB 52.2 MB/s 
     |████████████████████████████████| 94 kB 2.0 MB/s 
     |████████████████████████████████| 271 kB 18.9 MB/s 
     |████████████████████████████████| 144 kB 49.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 4.0 MB 4.8 MB/s 
     |████████████████████████████████| 880 kB 44.7 MB/s

In [ ]:
!git clone https://github.com/H-TayyarMadabushi/SemEval_2022_Task2-idiomaticity.git

Cloning into 'SemEval_2022_Task2-idiomaticity'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 123 (delta 48), reused 61 (delta 15), pack-reused 0
Receiving objects: 100% (123/123), 2.50 MiB | 10.13 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [ ]:
!git clone https://github.com/timoschick/bertram.git
!git clone https://github.com/timoschick/form-context-model.git

fatal: destination path 'bertram' already exists and is not an empty directory.
fatal: destination path 'form-context-model' already exists and is not an empty directory.


In [ ]:
import re
import os
import sys
import csv
import gzip
import math
import torch
import random
import numpy as np

from datetime                         import datetime
from torch.utils.data                 import DataLoader
from sklearn.metrics.pairwise         import paired_cosine_distances

from datasets                         import load_dataset
from transformers                     import AutoModelForMaskedLM
from transformers                     import AutoTokenizer
from sentence_transformers            import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.readers    import InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

sys.path.append( '/content/SemEval_2022_Task2-idiomaticity/SubTaskB/' )
sys.path.append('bertram/')
sys.path.append('form-context-model/')
from SubTask2Evaluator                import evaluate_submission

In [ ]:
languages=['PT']
def _parse_train_data( train_data_location, languages, tokenize=True ) :

  header, train_data = load_csv( train_data_location )
  
  train_data_with_labels                 = list()
  train_data_requiring_labels            = list()
  need_predictions_for_train_data_labels = list()

  # ['ID', 'MWE1', 'MWE2', 'Language', 'sentence_1', 'sentence_2', 'sim', 'alternative_1', 'alternative_2']

  skipped = 0 
  for elem in train_data :

    if not elem[ header.index( 'Language' ) ] in languages :
      skipped += 1
      continue

    mwe1          = elem[ header.index( 'MWE1'          ) ] 
    mwe2          = elem[ header.index( 'MWE2'          ) ] 
    
    this_sim      = elem[ header.index( 'sim'           ) ]
    sentence_1    = elem[ header.index( 'sentence_1'    ) ]
    sentence_2    = elem[ header.index( 'sentence_2'    ) ]
    alternative_1 = elem[ header.index( 'alternative_1' ) ]
    alternative_2 = elem[ header.index( 'alternative_2' ) ]

    ## Remove below if you do not want to tokenize with idiom tokens!
    if tokenize : 
      if mwe1 != 'None' : 
        replaced = re.sub( mwe1, tokenise_idiom( mwe1 ), sentence_1, flags=re.I)
        assert replaced != sentence_1
        sentence_1 = replaced
      if mwe2 != 'None' : 
        replaced = re.sub( mwe1, tokenise_idiom( mwe2 ), sentence_2, flags=re.I)
        assert replaced != sentence_2
        sentence_2 = replaced
  
   
    if this_sim != 'None' :
      tmp = float( this_sim ) 
      train_data_with_labels.append( [ sentence_1, sentence_2, this_sim ] ) 
      continue
    train_data_requiring_labels.append( [ sentence_1, sentence_2 ] ) 
    need_predictions_for_train_data_labels.append( [ alternative_1, alternative_2 ] )

  assert len( need_predictions_for_train_data_labels ) == len( train_data_requiring_labels )
  assert len( train_data ) == len( need_predictions_for_train_data_labels ) + len( train_data_with_labels ) + skipped

  return train_data_with_labels, train_data_requiring_labels, need_predictions_for_train_data_labels 

In [ ]:
def load_csv( path ) : 
  header = None
  data   = list()
  with open( path, encoding='utf-8') as csvfile:
    reader = csv.reader( csvfile ) 
    for row in reader : 
      if header is None : 
        header = row
        continue
      data.append( row ) 
  return header, data
def tokenise_idiom( phrase ) :
  return 'ID' + re.sub( r'[\s|-]', '', phrase ).lower() + 'ID'

In [ ]:
def prepare_eval_data( location, languages, test_print=False ) :
  header, data = load_csv( location )
  sentence1s = list()
  sentence2s = list()
  for elem in data : 
    if not languages is None and not elem[ header.index( 'Language' ) ] in languages : 
      continue
    sentence1 = elem[ header.index( 'sentence1' ) ] 
    sentence2 = elem[ header.index( 'sentence2' ) ] 
    mwe1      = elem[ header.index( 'MWE1'      ) ] 
    mwe2      = elem[ header.index( 'MWE2'      ) ] 

    if test_print : 
      print( sentence1 ) 
      print( sentence2 ) 
      print( mwe1 ) 
      print( mwe2 ) 

    if mwe1 != 'None' : 
      replaced = re.sub( mwe1, tokenise_idiom( mwe1 ), sentence1, flags=re.I)
      assert replaced != sentence1
      sentence1 = replaced
    if mwe2 != 'None' : 
      replaced = re.sub( mwe1, tokenise_idiom( mwe2 ), sentence2, flags=re.I)
      assert replaced != sentence2
      sentence2 = replaced

    if test_print : 
      print( sentence1 ) 
      print( sentence2 ) 
      break

    sentence1s.append( sentence1 ) 
    sentence2s.append( sentence2 ) 

  return sentence1s, sentence2s


def get_similarities( location, model, languages=None ) : 
  sentences1, sentences2 = prepare_eval_data( location, languages ) 

  #Compute embedding for both lists
  embeddings1 = model.encode(sentences1, show_progress_bar=True, convert_to_numpy=True)
  embeddings2 = model.encode(sentences2, show_progress_bar=True, convert_to_numpy=True)

  # Compute cosine-similarits
  cosine_scores = 1 - (paired_cosine_distances(embeddings1, embeddings2))

  return cosine_scores

In [ ]:
def insert_to_submission( languages, settings, sims, location ) : 
  header, data = load_csv( location ) 
  sims = list( reversed( sims ) )
  ## Validate with length
  updatable = [ i for i in data if i[ header.index( 'Language' ) ] in languages and i[ header.index( 'Setting' ) ] in settings ]
  assert len( updatable ) == len( sims ) 

  ## Will update in sequence - if data is not in sequence must update one language / setting at a time. 
  started_update = False
  for elem in data : 
    if elem[ header.index( 'Language' ) ] in languages and elem[ header.index( 'Setting' ) ] in settings : 
      sim_to_insert = sims.pop()
      elem[-1] = sim_to_insert
      started_update = True
    else :  
      assert not started_update ## Once we start, we must complete. 
    if len( sims ) == 0 : 
      break 
  assert len( sims ) == 0 ## Should be done here. 

  return [ header ] + data

def write_csv( data, location ) : 
  with open( location, 'w', encoding='utf-8') as csvfile:
    writer = csv.writer( csvfile ) 
    writer.writerows( data ) 
  print( "Wrote {}".format( location ) ) 
  return

In [ ]:
def _get_predictions_for_train_data_labels( model_path, data ) :

  model      = SentenceTransformer( model_path )

  sentences1 = [ i[0] for i in data ]
  sentences2 = [ i[1] for i in data ]

  embeddings1 = model.encode(sentences1, show_progress_bar=True, convert_to_numpy=True)
  embeddings2 = model.encode(sentences2, show_progress_bar=True, convert_to_numpy=True)

  cosine_scores = 1 - (paired_cosine_distances(embeddings1, embeddings2))

  return cosine_scores

def generate_train_data( train_data_location, model_path, languages ) :
  
  train_data_with_labels, train_data_requiring_labels, need_predictions_for_train_data_labels = _parse_train_data( train_data_location, languages )
  sims = _get_predictions_for_train_data_labels( model_path, need_predictions_for_train_data_labels )

  train_data_requiring_labels_with_labels = list()
  for index in range( len( train_data_requiring_labels ) ) : 
    train_data_requiring_labels_with_labels.append( [ train_data_requiring_labels[index][0], train_data_requiring_labels[index][1], sims[index] ] )

  train_data = [ [ 'sentence_1', 'sentence_2', 'sim' ] ] + train_data_with_labels + train_data_requiring_labels_with_labels
  assert all( [ (len(i) == 3) for i in train_data ] )
  
  return train_data

In [ ]:
data_location = '/content/SemEval_2022_Task2-idiomaticity/SubTaskB/EvaluationData/'
outpath = '/content/models/'
dev_location                = os.path.join( data_location, 'dev.csv'                     ) 
eval_location               = os.path.join( data_location, 'eval.csv'                    ) 
dev_formated_file_location  = os.path.join( data_location, 'dev.submission_format.csv'   ) 
## Save tmp model here.  
outdir = os.path.join( outpath, 'pBERT' + '-' + str( 4 ) ) 

In [ ]:
!mkdir models

In [ ]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
train_data_location = 'SemEval_2022_Task2-idiomaticity/SubTaskB/TrainData/train_data.csv'

In [ ]:
train_data_with_labels, train_data_requiring_labels, need_predictions_for_train_data_labels = _parse_train_data( train_data_location, languages )

In [ ]:
idioms = list()
for data_split in [ 'dev', 'eval' ] : 
  file_path = os.path.join( data_location, data_split + '.csv' )
  header, data = load_csv( file_path )
  for elem in data : 
    if elem[ header.index( 'Language' ) ] =='PT' :
      idioms.append( elem[ header.index( 'MWE1' ) ] )
      idioms.append( elem[ header.index( 'MWE2' ) ] )

idioms = list( set( idioms ) ) 
idioms.remove( 'None' )

In [ ]:
idioms = [ tokenise_idiom( i ) for i in idioms ]

## **More Training data**

### Pcocessing training data for bertram

In [ ]:
!wget https://data.statmt.org/cc-100/pt.txt.xz

--2022-05-07 02:43:04--  https://data.statmt.org/cc-100/pt.txt.xz
Resolving data.statmt.org (data.statmt.org)... 129.215.197.184
Connecting to data.statmt.org (data.statmt.org)|129.215.197.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13677690424 (13G) [application/x-xz]
Saving to: ‘pt.txt.xz’

pt.txt.xz             0%[                    ]  21.49M  6.57MB/s    eta 33m 54s

In [ ]:
!wget https://data.statmt.org/cc-100/gl.txt.xz

In [ ]:
!apt-get install xz-utils

In [ ]:
!xz -d gl.txt.xz

In [ ]:
corpula=[]
num=0
with open('gl.txt','r') as f:
  for line in f.readlines():
    if num==1000000:
      break
    corpula.append(line)
    num+=1
f.close()
f=open("train_corpus_l.txt","w")
f.writelines(corpula)
f.close()

In [ ]:
!mkdir train_data

In [ ]:
!python form-context-model/fcm/preprocess.py train --input train_corpus_l.txt --output train_data/

In [ ]:
!mkdir train_data/buckets
!mv train_data/train.bucket* train_data/buckets/

In [ ]:
# some code change in the train.py() get_linear_schedule_with_warmup num_warmup_steps num_training_steps and exchange the sequence for betram.py line 193
!python3 bertram/train.py \
   --model_cls bert \
   --bert_model bert-base-uncased \
   --output_dir outputs/gl_1m \
   --train_dir train_data/buckets/ \
   --vocab train_data/train.vwc100 \
   --emb_file embeddings-bert-base-uncased.txt \
   --num_train_epochs 2 \
   --emb_dim 768 \
   --train_batch_size 16 \
   --smin 1 \
   --smax 1 \
   --max_seq_length 32 \
   --mode form \
   --learning_rate 0.01 \
   --dropout 0.1 \

#### move betram to drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
# !mkdir outputs
# !cp -r /content/gdrive/MyDrive/idiom_detection/models/bertram/* /content/outputs

In [ ]:

# !mkdir -p /content/gdrive/MyDrive/idiom_detection/models/bertram
# !cp -r /content/outputs/* /content/gdrive/MyDrive/idiom_detection/models/bertram

### Incorporate bertram embedding

In [ ]:
from bertram import BertramWrapper

In [ ]:
bertram = BertramWrapper('outputs/gl-e1', device='cpu')

In [ ]:
model_name='neuralmind/bert-base-portuguese-cased'
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, truncation=True)

In [ ]:
words_with_contexts ={}
for i in idioms:
  words_with_contexts[i]=[]
embeddings={word: bertram.infer_vector(word, contexts) for word, contexts in words_with_contexts.items()}

In [ ]:
special_tokens=[f"<BERTRAM:{word}>" for word in embeddings.keys()]
tokenizer.add_special_tokens({"additional_special_tokens": special_tokens})

In [ ]:
embedding_weight = model.bert.embeddings.word_embeddings.weight
max_id =max(tokenizer.additional_special_tokens_ids)
model.resize_token_embeddings(len(tokenizer))

In [ ]:
model.bert.embeddings.word_embeddings.weight.requires_grad=False

In [ ]:
for word, embedding in embeddings.items():
      word_id = tokenizer.convert_tokens_to_ids(f"<BERTRAM:{word}>")
      model.bert.embeddings.word_embeddings.weight[word_id] = embedding

In [ ]:
model.save_pretrained( outdir )
tokenizer.save_pretrained(outdir)

### Pre_train model

In [ ]:
train_samples = []
dev_samples   = []
test_samples  = []
for split in [ 'train', 'validation', 'test' ] :
      dataset = load_dataset( 'assin2', split=split, ignore_verifications=True )
      for elem in dataset :
        ## {'entailment_judgment': 1, 'hypothesis': 'Uma criança está segurando uma pistola de água', 'premise': 'Uma criança risonha está segurando uma pistola de água e sendo espirrada com água', 'relatedness_score': 4.5, 'sentence_pair_id': 1}
          score = float( elem['relatedness_score'] ) / 5.0 # Normalize score to range 0 ... 1
          inp_example = InputExample(texts=[elem['hypothesis'], elem['premise']], label=score)
          if split == 'validation':
            dev_samples.append(inp_example)
          elif split == 'test':
            test_samples.append(inp_example)
          elif split == 'train' :
            train_samples.append(inp_example)
          else :
              raise Exception( "Unknown split. Should be one of ['train', 'test', 'validation']." )

In [ ]:
model_path = outdir
sent_trans_path='models/psentBert-4/'
word_embedding_model = models.Transformer(model_path)

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                                pooling_mode_mean_tokens=True,
                                pooling_mode_cls_token=False,
                                pooling_mode_max_tokens=False)


tokenizer      = AutoTokenizer.from_pretrained(
  model_path             , 
  use_fast       = False ,
  max_length     = 510   ,
  force_download = True
)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
model._first_module().tokenizer = tokenizer

model.save( sent_trans_path )


In [ ]:
train_batch_size = 8
num_epochs       = 2
model_save_path='models/PnosentBert-8'
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
train_loss       = losses.CosineSimilarityLoss(model=model)
  
evaluator        = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')

# Configure the training. 
warmup_steps     = math.ceil(len(train_dataloader) * num_epochs  * 0.1) #10% of train data for warm-up
print("Warmup-steps: {}".format(warmup_steps), flush=True)

# Train the model

model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path
)

test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
test_evaluator(model, output_path=model_save_path)

model_path = model_save_path

In [ ]:
model = SentenceTransformer('models/PnosentBert-8')
dev_sims  = get_similarities( dev_location , model, languages ) 

In [ ]:
## Create submission file on the development set. 
submission_data = insert_to_submission( languages, [ 'pre_train' ], dev_sims, dev_formated_file_location )  
results_file    = os.path.join( outpath, 'dev.pre_train_results-' + str( 4 ) + '.csv' )
write_csv( submission_data, results_file )

## Evaluate development set. 
results = evaluate_submission( results_file, os.path.join( data_location, 'dev.gold.csv' ) )

In [ ]:
results

### Fine_tune

In [ ]:
# !mkdir gdrive/MyDrive/idiom_detection/models/large
# !cp models/PnosentBert-8/* gdrive/MyDrive/idiom_detection/models/large/

In [ ]:
best_pre_train_seed = 4 ## Found this by running above (as in pre-train setting) multiple times. 

train_data_location = 'SemEval_2022_Task2-idiomaticity/SubTaskB/TrainData/train_data.csv'
out_location        = 'models/FineTune/'

model_path = '/content/models/PnosentBert-8/'
train_data = generate_train_data( train_data_location, model_path, languages )

In [ ]:
def is_torch_available() :
    try:
        import torch
        return True
    except ImportError:
        return False

def is_tf_available() :
    try:
        import tensorflow as tf
        return True
    except ImportError:
        return False
def set_seed(seed: int):
  """
  Modified from : https://github.com/huggingface/transformers/blob/master/src/transformers/trainer_utils.py
  Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
  installed).
  Args:
      seed (:obj:`int`): The seed to set.
  """
  random.seed(seed)
  np.random.seed(seed)
  if is_torch_available():
      torch.manual_seed(seed)
      torch.cuda.manual_seed_all(seed)
      # ^^ safe to call this function even if cuda is not available

      ## From https://pytorch.org/docs/stable/notes/randomness.html
      torch.backends.cudnn.benchmark = False

      ## Might want to use the following, but set CUBLAS_WORKSPACE_CONFIG=:16:8
      # try : 
      #   torch.use_deterministic_algorithms(True)
      # except AttributeError: 
      #   torch.set_deterministic( True )
      
  if is_tf_available():
      import tensorflow as tf
      tf.random.set_seed(seed)

In [ ]:
def create_and_eval_subtask_b_fine_tune( 
    model_path, 
    seed, 
    data_location, 
    dev_formated_file_location,
    eval_formated_file_location,
    train_data, 
    out_location, 
    languages, 
    epoch=None 
    ) :

  set_seed( seed )
  
  dev_location                = os.path.join( data_location, 'dev.csv'                     ) 
  eval_location               = os.path.join( data_location, 'eval.csv'                    ) 


  ## Training Dataloader
  train_samples = list()

  header     = train_data[0] ## ['sentence_1', 'sentence_2', 'sim']
  train_data = train_data[1:]
  for elem in train_data :
    score = float( elem[2] ) 
    inp_example = InputExample(texts=[elem[0], elem[1]], label=score)
    train_samples.append(inp_example)


  ## Params
  train_batch_size = 4
    
  model            = SentenceTransformer( model_path )
  train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
  train_loss       = losses.CosineSimilarityLoss(model=model)
  
 # Train the model
  dev_sims = eval_sims = results = None
  if epoch is None :
    ## Going to test all epochs - notice we can't use the default evaluator. 
    for epoch in range( 1, 10 ) :
      warmup_steps     = math.ceil(len(train_dataloader) * epoch  * 0.1) #10% of train data for warm-up
      print("Warmup-steps: {}".format(warmup_steps), flush=True)
      
      model_save_path = os.path.join( out_location, str( seed ), str( epoch ) ) 
      model.fit(train_objectives=[(train_dataloader, train_loss)],
                evaluator=None,
                epochs=1,
                evaluation_steps=0,
                warmup_steps=warmup_steps,
                output_path=model_save_path
      )

      dev_sims  = get_similarities( dev_location , model, languages ) 
      eval_sims = get_similarities( eval_location, model, languages )

      ## Create submission file on the development set. 
      submission_data = insert_to_submission( languages, [ 'fine_tune' ], dev_sims, dev_formated_file_location )  
      results_file    = os.path.join( outpath, 'dev.combined_results-' + str( seed ) + '.csv' )
      write_csv( submission_data, results_file )

      ## Evaluate development set. 
      results = evaluate_submission( results_file, os.path.join( data_location, 'dev.gold.csv' ) )

      ## Make results printable. 
      for result in results : 
        for result_index in range( 2, 5 ) : 
          result[result_index] = 'Did Not Attempt' if result[result_index] is None else result[ result_index ]
  
      for row in results : 
        print( '\t'.join( [str(i) for i in row ] ) )
        
      results_file = os.path.join( model_save_path, 'RESULTS_TABLE-dev.pre_train_' + str(epoch) + str( seed ) + '.csv' )    
      write_csv( results, results_file )      

      ## Generate combined output for this epoch.
      submission_data = insert_to_submission( languages, [ 'fine_tune' ], eval_sims, eval_formated_file_location )  
      results_file    = os.path.join( outpath, 'eval.combined_results-' + str( seed ) + '_' + str( epoch ) + '.csv' )
      write_csv( submission_data, results_file )

 
  else :
    ## We already know the best epoch and so will use it.
    warmup_steps     = math.ceil(len(train_dataloader) * epoch  * 0.1) #10% of train data for warm-up
    print("Warmup-steps: {}".format(warmup_steps), flush=True)

    model_save_path = os.path.join( out_location, str( seed ), str( epoch ) ) 
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              evaluator=None,
              epochs=epoch,
              evaluation_steps=0,
              warmup_steps=warmup_steps,
              output_path=model_save_path
    )
    
    dev_sims  = get_similarities( dev_location , model, languages ) 
    eval_sims = get_similarities( eval_location, model, languages )

    ## Create submission file on the development set. 
    submission_data = insert_to_submission( languages, [ 'fine_tune' ], dev_sims, dev_formated_file_location )  
    results_file    = os.path.join( outpath, 'dev.combined_results-' + str( seed ) + '.csv' )
    write_csv( submission_data, results_file )
    
    ## Evaluate development set. 
    results = evaluate_submission( results_file, os.path.join( data_location, 'dev.gold.csv' ) )
    
    ## Make results printable. 
    # for result in results : 
    #   for result_index in range( 2, 5 ) : 
    #     result[result_index] = 'Did Not Attempt' if result[result_index] is None else result[ result_index ]
  
    # results_file = os.path.join( model_save_path, 'RESULTS_TABLE-dev.pre_train_' + str(epoch) + str( seed ) + '.csv' )    
    # write_csv( results, results_file )



  ## Outside if
  return results

In [ ]:
params = {
    'model_path'                  : model_path, 
    'seed'                        : 4, 
    'data_location'               : data_location, 
    'dev_formated_file_location'  : '/content/models/dev.pre_train_results-4.csv',  ## We can append to this.
    'eval_formated_file_location' : '/content/models/eval.pre_train_results-4.csv',
    'train_data'                  : train_data , 
    'out_location'                : out_location ,  
    'languages'                   : languages ,
    'epoch'                       : 4 ,
} 

In [ ]:
results=create_and_eval_subtask_b_fine_tune(**params)

In [ ]:
results